In [1]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from nltk import word_tokenize
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

Using TensorFlow backend.


In [2]:
class Cbow:
    def __init__(self, text: str, context_window: int=1):
        self._text = text
        self._tokens = word_tokenize(text)
        self._len_tokens = len(self._tokens)
        self._context_window = context_window
        self.id2word = {
            _id+1: text 
            for _id, text in enumerate(set(self.tokens))
        }
        self.id2word[0] = "PAD"
        self.vocab_size = len(self.id2word)
        self.word2id = {v: k for k, v in self.id2word.items()}
        
    @property
    def text(self):
        return self._text
    
    @property
    def len_tokens(self):
        return self._len_tokens
    
    @property
    def tokens(self):
        return self._tokens
    
    @property
    def context_window(self):
        return self._context_window
    
    def get_context_words(self):
        for index, word in enumerate(self.tokens):
                        
            start = index - self.context_window
            end = index + self.context_window + 1
            
            if index - self.context_window < 0:
                continue
            elif index + self.context_window == self.len_tokens:
                break
                
            word_list = [
                self.tokens[i] for i in range(start, end)
                if i != index
            ]
            
            yield (
                np.array([1 if index == i else 0 for i in range(self.vocab_size)]).reshape(1, self.vocab_size), 
                np.array([self.word2id[word] for word in word_list]).reshape(1,2)
            )

    def train(self, epochs=10):
        vocab_size = len(self.id2word)
        embed_size = 100
        cbow = Sequential()
        cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=self._context_window*2))
        cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
        cbow.add(Dense(vocab_size, activation='softmax'))
        cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')
        
        for epoch in range(epochs):
            loss = 0.
            i = 0
            for y, x in self.get_context_words():
                i += 1
                loss += cbow.train_on_batch(x, y)
                if i % 1000 == 0:
                    print('Processed {} (context, word) pairs'.format(i))

            print('Epoch:', epoch, '\tLoss:', loss)
            print()
        
        weights = cbow.get_weights()[0]
        weights = weights[1:]
        
        return weights

    def __repr__(self):
        return str(self.tokens)

In [3]:
texto_dom_casmurro = """
Publicado pela primeira vez em 1899, “Dom Casmurro” é uma das grandes obras de Machado de Assis e confirma o olhar certeiro e crítico que o autor estendia sobre toda a sociedade brasileira. Também a temática do ciúme, abordada com brilhantismo nesse livro, provoca polêmicas em torno do caráter de uma das principais personagens femininas da literatura brasileira: Capitu.
O romance inicia-se numa situação posterior a todos os seus acontecimentos. Bento Santiago, já um homem de idade, conta ao leitor como recebeu a alcunha de Dom Casmurro. A expressão fora inventada por um jovem poeta, que tentara ler para ele no trem alguns de seus versos. Como Bento cochilara durante a leitura, o rapaz ficou chateado e começou a chamá-lo daquela forma.
Bentinho vai estudar direito no Largo de São Francisco, em São Paulo. Quando conclui os estudos, torna-se o doutor Bento de Albuquerque Santiago. Ocorre então o casamento tão esperado entre Bento e Capitu. Escobar, por seu lado, casara-se com Sancha, uma antiga amiga de colégio de Capitu. Capitu e Bentinho formam um “duo afinadíssimo”.

Essa felicidade, entretanto, começa a ser ameaçada com a demora do casal em ter um filho. Escobar e Sancha não encontram a mesma dificuldade: têm uma bela menina, a quem colocam o nome de Capitolina.

Depois de alguns anos, Capitu finalmente tem um filho, e o casal pode retribuir a homenagem que Escobar e Sancha lhe haviam prestado: o filho é batizado com o nome de Ezequiel.

Os casais passam a conviver intensamente. Bento vê uma semelhança terrível entre o pequeno Ezequiel e seu amigo Escobar, que, numa de suas aventuras na praia – o personagem era excelente nadador -, morre afogado.

Bento enxerga no filho a figura do amigo falecido e fica convencido de que fora traído pela mulher. Resolve suicidar-se bebendo uma xícara de café envenenado. Quando Ezequiel entra em seu escritório, decide matar a criança, mas desiste no último momento. Diz ao garoto, então, que não é seu pai. Capitu escuta tudo e lamenta-se pelo ciúme de Bentinho, que, segundo ela, fora despertado pela casualidade da semelhança.

Após inúmeras discussões, o casal decide separar-se. Arruma-se uma viagem para a Europa com o intuito de encobrir a nova situação, que levantaria muita polêmica. O protagonista retorna sozinho ao Brasil e se torna, pouco a pouco, o amargo Dom Casmurro. Capitu morre no exterior e Ezequiel tenta reatar relações com ele, mas a semelhança extrema com Escobar faz com que Bento Santiago o rejeite novamente. O destino de Ezequiel é infeliz: ele morre de febre tifóide durante uma pesquisa arqueológica em Jerusalém.

Triste e nostálgico, o narrador constrói uma casa que imita sua casa de infância, na rua de Matacavalos. O próprio livro é também uma tentativa de recuperar o sentido de sua vida. No fim, o narrador parece menosprezar um pouco a própria criação. Convence-se de que o melhor a fazer agora é escrever outra obra sobre “a história dos subúrbios”.

Lista de personagens
Bentinho (Bento Santiago): o narrador-personagem que conta suas memórias, membro da elite carioca do século XIX.

Capitu (Capitolina): grande amor de Bentinho, personagem de origem pobre, mas independente e avançada.

Escobar: o melhor amigo de Bentinho, a quem conheceu quando estudaram juntos no seminário.

Dona Sancha: mulher de Escobar, ex-colega de colégio de Capitu.

Dona Glória: mãe de Bentinho, adora o filho e é também muito religiosa. Quer que o garoto se ordene padre como cumprimento de uma promessa que fez.

José Dias: agregado que vive de favores na casa de dona Glória. Suposto médico, tem o hábito de agradar aos proprietários da casa com o uso de superlativos.

Tio Cosme: irmão de dona Glória, viúvo e advogado.

Prima Justina: prima de dona Glória, que, segundo o narrador, não tinha papas na língua.

Pedro de Albuquerque Santiago: pai de Bentinho, faleceu quando o filho ainda era muito pequeno.

Senhor Pádua e Dona Fortunata: pais de Capitu, que viam no possível casamento da filha com Bentinho uma possibilidade de ascensão social.

Ezequiel: filho de Capitu, sobre o qual o narrador sustenta forte dúvida quanto à paternidade, pois o garoto tinha grande semelhança física com Escobar.

Sobre Machado de Assis
Joaquim Maria Machado de Assis nasceu em 21 de junho de 1839 na cidade do Rio de Janeiro. Neto de escravos alforriados, foi criado em uma família pobre e não pode frequentar regularmente a escola. Porém, devido a seu enorme interesse por literatura, conseguiu se instruir por conta própria. Entre os seis e os quatorze anos, Machado de Assis perdeu sua irmã, a mãe e o pai.

Aos 16 anos, Machado conseguiu um emprego como aprendiz em uma tipografia, vindo a publicar seus primeiros versos no jornal “A Marmota”. Em 1860 passou a colaborar para o “Diário do Rio de Janeiro” e é dessa década que datam quase todas suas comédias teatrais e “Crisálidas”, um livro de poemas.

Em 1869 Machado de Assis casou-se com Carolina Augusta Xavier de Novais sem o consentimento da família da moça, devido à má fama que Machado carregava. Porém, este casamento mudou sua vida, uma vez que Carolina lhe apresentou à literatura portuguesa e inglesa. Mais amadurecido literariamente, Machado publica na década de 1870 uma série de romances, tais como “A mão e a luva” (1874) e “Helena” (1876), vindo a obter reconhecimento do público e da crítica. Ainda na década de 1870, Machado iniciou sua carreira burocrática e em 1892 já ocupava o cargo de diretor geral do Ministério da Aviação. Através de sua carreira no serviço público, Machado de Assis conseguiu sua estabilidade financeira.

A obra literária de Machado era marcadamente romântica, mas na década de 1880 ela sofre uma grande mudança estilística e temática, vindo a inaugurar o Realismo no Brasil com a publicação de “Memórias Póstumas de Brás Cubas” (1881). A partir de então a ironia, o pessimismo, o espírito crítico e uma profunda reflexão sobre a sociedade brasileira se tornarão as principais características de suas obras. Em 1897, Machado funda a Academia Brasileira de Letras, sendo seu primeiro presidente e ocupando a Cadeira Nº 23.

Em 1904, Machado perde a esposa após um casamento de 35 anos. A morte de Carolina abalou profundamente o escritor, que passou a ficar isolado em casa e sua saúde foi piorando. Dessa época datam seus últimos romances: “Esaú e Jacó” (1904) e “Memorial de Aires” (1908). Machado morreu em sua casa no Rio de Janeiro no dia 29 de setembro de 1908. Seu enterro foi acompanhado por uma multidão e foi decretado luto oficial no Rio de Janeiro.

Seus principais romances são: “Ressurreição” (1872), “A mão e a luva” (1874), “Helena” (1876), “Iaiá Garcia” (1878), “Memórias Póstumas de Brás Cubas” (1881), “Quincas Borba” (1891), “Dom Casmurro” (1899), “Esaú e Jacó” (1904) e “Memorial de Aires” (1908). Além dessas obras, Machado de Assis possui uma extensa bibliografia que abrange poemas, contos e peças teatrais.
"""

cbow = Cbow(texto_dom_casmurro, context_window=1)

In [4]:
w = cbow.train(100)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.
Processed 1000 (context, word) pairs
Epoch: 0 	Loss: 3559.6175861358643

Processed 1000 (context, word) pairs
Epoch: 1 	Loss: 3547.8311491012573

Processed 1000 (context, word) pairs
Epoch: 2 	Loss: 3535.2587208747864

Processed 1000 (context, word) pairs
Epoch: 3 	Loss: 3520.5795307159424

Processed 1000 (context, word) pairs
Epoch: 4 	Loss: 3503.6844730377197

Processed 1000 (context, word) pairs
Epoch: 5 	Loss: 3484.717423439026

Processed 1000 (context, word) pairs
Epoch: 6 	Loss: 3463.9010195732117

Processed 1000 (context, word) pairs
Epoch: 7 	Loss: 3441.3661575317383

Processed 1000 (context, word) pairs
Epoch: 8 	Loss: 3417.190248966217

Processed 1000 (context, word) pairs
Epoch: 9 	Loss: 3391.2923765182495

Processed 1000 (context, word) pairs
Epoch: 10 	Loss: 3363.6668181419373

Proce

KeyboardInterrupt: 

In [ ]:
# compute pairwise distance matrix
distance_matrix = euclidean_distances(w)
print(distance_matrix.shape)

# view contextually similar words
similar_words = {search_term: [cbow.id2word[idx] for idx in distance_matrix[cbow.word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['filho', 'romances', 'mão', 'luva']}

similar_words